# ![IronHack Logo](https://s3-eu-west-1.amazonaws.com/ih-materials/uploads/upload_d5c5793015fec3be28a63c4fa3dd4d55.png)

# Lab | Advanced MySQL

## Introduction

In this lab you will practice SQL Subqueries and Action Queries. We will again use the publications database.

Create a `solutions.ipynb` file in the `your-code` directory to record your solutions to all challenges.

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/MarLizana/Desktop/IH/ironhack_service_account_big_query.json"
from google.cloud import bigquery
client = bigquery.Client()

# Challenge 1 - Most Profiting Authors

In this challenge let's have a close look at the bonus challenge of the previous *SQL SELECT* lab -- **who are the top 3 most profiting authors**? Even if you have solved or think you have solved that problem in the previous lab, please still complete this challenge because the step-by-step guidances are helpful to train your problem-solving thinking.

In order to solve this problem, it is important for you to keep the following points in mind:

* In table `sales`, a title can appear several times. The royalties need to be calculated for each sale.

* Despite a title can have multiple `sales` records, the advance must be calculated only once for each title.

* In your eventual solution, you need to sum up the following profits for each individual author:
    * All advances which is calculated exactly once for each title.
    * All royalties in each sale.

Therefore, you will not be able to achieve the goal with a single SELECT query. Instead, you will need to follow several steps in order to achieve the eventual solution. Below is an overview of the steps:

1. Calculate the royalty of each sale for each author.

1. Using the output from Step 1 as a sub-table, aggregate the total royalties for each title for each author.

1. Using the output from Step 2 as a sub-table, calculate the total profits of each author by aggregating the advances and total royalties of each title.

Below we'll guide you through each step. In your `solutions.ipynb`, please include the SELECT queries of each step so that your TA can review your problem-solving process.

# Step 1: Calculate the royalties of each sales for each author

Write a SELECT query to obtain the following output:

* Title ID
* Author ID
* Royalty of each sale for each author
    * The formula is:
        ```
        sales_royalty = titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100
        ```
    * Note that `titles.royalty` and `titleauthor.royaltyper` are divided by 100 respectively because they are percentage numbers instead of floats.

In the output of this step, each title may appear more than once for each author. This is because a title can have more than one sales.

In [6]:
query_c1_s1 = """
SELECT 
    t.title_id AS title_id,
    a.au_id AS author_id,
    t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100 AS sales_royalty
    
FROM 
    `ironhack-data-analytics-265219.publications.sales` AS s
INNER JOIN
    `ironhack-data-analytics-265219.publications.titles` AS t
ON 
    s.title_id = t.title_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.titleauthor` AS ta
ON
    ta.title_id = s.title_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.authors` AS a
ON
    a.au_id = ta.au_id
ORDER BY
    1, 2

"""

query_job = client.query(query=query_c1_s1)

In [7]:
query_job.to_dataframe()

,title_id,author_id,sales_royalty
0,BU1032,213-46-8915,3.998
1,BU1032,213-46-8915,7.996
2,BU1032,409-56-7008,5.997
3,BU1032,409-56-7008,11.994
4,BU1111,267-41-2394,11.950
5,BU1111,724-80-9391,17.925
6,BU2075,213-46-8915,25.116
7,BU7832,274-80-9391,29.985
8,MC2222,712-45-1867,23.988
9,MC3021,722-51-5454,13.455


# Step 2: Aggregate the total royalties for each title for each author

Using the output from Step 1, write a query to obtain the following output:

* Title ID
* Author ID
* Aggregated royalties of each title for each author
    * Hint: use the *SUM* subquery and group by both `au_id` and `title_id`

In the output of this step, each title should appear only once for each author.


In [95]:
query_c1_s2 = """
SELECT 
    t.title_id AS title_id,
    a.au_id AS author_id,
    SUM(t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100) AS sales_royalty
    
FROM 
    `ironhack-data-analytics-265219.publications.sales` AS s
INNER JOIN
    `ironhack-data-analytics-265219.publications.titles` AS t
ON 
    s.title_id = t.title_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.titleauthor` AS ta
ON
    ta.title_id = s.title_id
INNER JOIN
    `ironhack-data-analytics-265219.publications.authors` AS a
ON
    a.au_id = ta.au_id
GROUP BY
    1,2
ORDER BY 
    1

"""
query_job = client.query(query=query_c1_s2)

In [96]:
query_job.to_dataframe()

,title_id,author_id,sales_royalty
0,BU1032,213-46-8915,11.994
1,BU1032,409-56-7008,17.991
2,BU1111,267-41-2394,11.950
3,BU1111,724-80-9391,17.925
4,BU2075,213-46-8915,25.116
5,BU7832,274-80-9391,29.985
6,MC2222,712-45-1867,23.988
7,MC3021,899-46-2035,7.176
8,MC3021,722-51-5454,21.528
9,PC1035,238-95-7766,110.160


# Step 3: Calculate the total profits of each author

Now that each title has exactly one row for each author where the advance and royalties are available, we are ready to obtain the eventual output. Using the output from Step 2, write a query to obtain the following output:

* Author ID
* Profits of each author by aggregating the advance and total royalties of each title

Sort the output based on a total profits from high to low, and limit the number of rows to 3.

In [110]:
query_c1_s3 = """
SELECT 
    author_id,
    SUM(summary.sales_royalty + summary.Advance)
    
FROM(
    SELECT 
        t.title_id AS title_id,
        a.au_id AS author_id,
        t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100 AS sales_royalty,
        t.advance * ta.royaltyper/100 AS Advance

    FROM 
        `ironhack-data-analytics-265219.publications.sales` AS s
    INNER JOIN
        `ironhack-data-analytics-265219.publications.titles` AS t
    ON 
        s.title_id = t.title_id
    INNER JOIN
        `ironhack-data-analytics-265219.publications.titleauthor` AS ta
    ON
        ta.title_id = s.title_id
    INNER JOIN
        `ironhack-data-analytics-265219.publications.authors` AS a
    ON
        a.au_id = ta.au_id
        ) summary
    
LEFT JOIN
    `ironhack-data-analytics-265219.publications.titles` AS t
ON 
    summary.title_id = t.title_id
    
LEFT JOIN
    `ironhack-data-analytics-265219.publications.titleauthor` AS ta
ON
     summary.author_id=ta.au_id and summary.title_id=ta.title_id

GROUP BY
    1

ORDER BY
    2 DESC

LIMIT 3

"""

query_job = client.query(query=query_c1_s3)

In [111]:
query_job.to_dataframe()

,author_id,f0_
0,722-51-5454,22521.528
1,213-46-8915,14162.110
2,899-46-2035,12128.132


# Elevating from your solution in Challenge 1 , create a table named `most_profiting_authors` to hold the data about the most profiting authors. The table should have 2 columns:

* `au_id` - Author ID
* `profits` - The profits of the author aggregating the advances and royalties

Include your solution in `solutions.ipynb`.

In [121]:
sql = '''
CREATE TABLE 
    `ironhack-data-analytics-265219.publications.most_profiting_authors3`
AS
(
SELECT 
    author_id,
    SUM(summary.sales_royalty + summary.Advance)
    
FROM(
    SELECT 
        t.title_id AS title_id,
        a.au_id AS author_id,
        t.price * s.qty * t.royalty / 100 * ta.royaltyper / 100 AS sales_royalty,
        t.advance * ta.royaltyper/100 AS Advance

    FROM 
        `ironhack-data-analytics-265219.publications.sales` AS s
    INNER JOIN
        `ironhack-data-analytics-265219.publications.titles` AS t
    ON 
        s.title_id = t.title_id
    INNER JOIN
        `ironhack-data-analytics-265219.publications.titleauthor` AS ta
    ON
        ta.title_id = s.title_id
    INNER JOIN
        `ironhack-data-analytics-265219.publications.authors` AS a
    ON
        a.au_id = ta.au_id
        ) summary
    
LEFT JOIN
    `ironhack-data-analytics-265219.publications.titles` AS t
ON 
    summary.title_id = t.title_id
    
LEFT JOIN
    `ironhack-data-analytics-265219.publications.titleauthor` AS ta
ON
     summary.author_id=ta.au_id and summary.title_id=ta.title_id

GROUP BY
    1

ORDER BY
    2 DESC

LIMIT 3

)

'''

In [122]:
query_job = client.query(query=sql)